In [1]:
import scipy.ndimage as ndi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import pims
#import time
#import csv
import skimage
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage import measure
import scipy.spatial
import seaborn as sns
%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d



In [3]:
#IN = pd.read_csv('./INPosition.csv', skiprows= 3)
#pqbp1 = pd.read_csv('./INPQBP1Position.csv', skiprows= 3)
#cgas = pd.read_csv('./INcGASPosition.csv', skiprows= 3)
#triple = pd.read_csv('./TriplePosition.csv', skiprows=3)

#f_pqbp1 = "/media/agulhas/N74DOMXTHP1/plate3wt/fix/tif/Imaris/PQBP1_Image_1_Statistics/Position.csv"
#f_IN = "/media/agulhas/N74DOMXTHP1/plate3wt/fix/tif/Imaris/IN_Image_1_Statistics/Position.csv"
#f_cgas = "/media/agulhas/N74DOMXTHP1/plate3wt/fix/tif/Imaris/cGAS_Image_1_Statistics/Position.csv"
#f_triple = "/media/agulhas/N74DOMXTHP1/plate3wt/fix/tif/Imaris/Triple_Image_1_Statistics/Position.csv"

df = pd.read_hdf('/home/jmamede/Data/D116N/20Dec2020/stored_data_trackpy_AG3.h5')
df_gfp  = pd.read_hdf('/home/jmamede/Data/D116N/20Dec2020/stored_data_trackpy_gfp.h5')
df_ruby = pd.read_hdf('/home/jmamede/Data/D116N/20Dec2020/stored_data_trackpy_ruby.h5')
#triple = pd.read_csv(f_triple, skiprows=3)
print(df['VisitP'].unique())
print(df['Filename'].unique())
print(df['PlateRow'].unique())

['10' '1' '2' '3' '4' '5' '6' '7' '8' '9']
['/home/jmamede/Data/D116N/VPRR3-D116_v10_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v1_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v2_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v3_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v4_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v5_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v6_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v7_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v8_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/VPRR3-D116_v9_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v10_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v1_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v2_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v3_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v4_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v5_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/mNeonD116_v6_PRJ.ome.tiff'
 '/home/jmamede/Data/D116N/

In [4]:
_what = 'mNeonD116'
AG3subset = df[df['PlateRow'] == _what]
GFPsubset = df_gfp[df_gfp['PlateRow'] == _what]
Ruby3subset = df_ruby[df_ruby['PlateRow'] == _what]

AG3subset = AG3subset[AG3subset['VisitP'] == '1']
GFPsubset = GFPsubset[GFPsubset['VisitP'] == '1']
Ruby3subset = Ruby3subset[Ruby3subset['VisitP'] == '1']

In [5]:
print(AG3subset['COMX mirfp670'])
AG3count = AG3subset.index.max()+1
print(GFPsubset['COMX GFP'])
GFPcount = GFPsubset.index.max()+1
print(Ruby3subset['COMX mRuby3'])
Ruby3count = Ruby3subset.index.max()+1

8055    107.123399
8056    990.039362
8057    703.996611
8058    527.076447
8059    753.026052
           ...    
9657    407.904187
9658    651.075732
9659    768.042945
9660    846.025840
9661     70.969364
Name: COMX mirfp670, Length: 1607, dtype: float64
2534    107.050859
2535    652.055556
2536    704.069174
2537    991.857753
2538    240.093498
           ...    
5452    768.999405
5453     70.095180
5454    180.026667
5455    791.985442
5456    846.967293
Name: COMX GFP, Length: 2923, dtype: float64
1007    799.950379
1008    907.015529
1009    912.888483
1010    918.954473
1011    241.947647
1012    248.915674
1013    168.919967
1014    122.946358
1015    839.995981
1016    651.015013
Name: COMX mRuby3, dtype: float64


In [19]:
%matplotlib qt5

fig = plt.figure()

psize = 0.220
# zsize = 0.125

# ax = plt.axes(projection='3d')
# ax.scatter3D(IN['Position X']*psize,IN['Position Y']*psize,IN['Position Z']*zsize, color='red')
# ax.scatter3D(pqbp1['Position X']*psize,pqbp1['Position Y']*psize,pqbp1['Position Z']*zsize, color='green')

ax = plt.axes()
ax.scatter(AG3subset['COMX mirfp670']*psize,AG3subset['COMY mirfp670']*psize, color='blue')

ax.scatter(GFPsubset['COMX GFP']*psize,GFPsubset['COMY GFP']*psize, color='green')
ax.scatter(Ruby3subset['COMX mRuby3']*psize,Ruby3subset['COMY mRuby3']*psize, color='red')
ax.axes.set_aspect('equal')

In [20]:
AG3subset = df
GFPsubset = df_gfp
Ruby3subset = df_ruby

# _what = 'mNeonD116'
# AG3subset = df[df['PlateRow'] == _what]
# GFPsubset = df_gfp[df_gfp['PlateRow'] == _what]
# Ruby3subset = df_ruby[df_ruby['PlateRow'] == _what]

# AG3subset = AG3subset[AG3subset['VisitP'] == '1']
# GFPsubset = GFPsubset[GFPsubset['VisitP'] == '1']
# Ruby3subset = Ruby3subset[Ruby3subset['VisitP'] == '1']
#
AG3subset['PlateRow'].unique()

array(['VPRR3-D116', 'mNeonD116', 'mWasabi-D116', 'optiR3-D117',
       'sfGFPD117'], dtype=object)

In [22]:
psize = 0.220
# zsize = 0.125

# AG3_closest = np.zeros((len(AG3subset['VisitP'].unique()))
all = {}
print("Row,Visit,AG3Count,FluoCount,Dual@500nm,PercentageDual")
for _what in df['PlateRow'].unique():
    AG3subset = df[df['PlateRow'] == _what]
    GFPsubset = df_gfp[df_gfp['PlateRow'] == _what]
    Ruby3subset = df_ruby[df_ruby['PlateRow'] == _what]
    visits = np.array([])
    for visit in AG3subset['VisitP'].unique():
        AG3visit = AG3subset[AG3subset['VisitP'] == visit]
        AG3count = len(AG3visit)
        
        AG3pos = list(zip(AG3visit['COMX mirfp670']*psize,AG3visit['COMY mirfp670']*psize))
        
        if _what == 'optiR3-D117':
            Ruby3visit = Ruby3subset[Ruby3subset['VisitP'] == visit]
            GFPpos = list(zip(Ruby3visit['COMX mRuby3']*psize,Ruby3visit['COMY mRuby3']*psize))
            Fluocount = len(Ruby3visit)
        elif _what == 'VPRR3-D116': 
            Ruby3visit = Ruby3subset[Ruby3subset['VisitP'] == visit]
            GFPpos = list(zip(Ruby3visit['COMX mRuby3']*psize,Ruby3visit['COMY mRuby3']*psize))
            Fluocount = len(Ruby3visit)
        else:
            GFPvisit = GFPsubset[GFPsubset['VisitP'] == visit]
            GFPpos = list(zip(GFPvisit['COMX GFP']*psize,GFPvisit['COMY GFP']*psize))
            Fluocount = len(GFPvisit)
    


        #Nearest Neighbor Brute Force - I used 5 pixels for max distance in imaris
        if _what != 'VPRR3-D116':
            GFP_kd = scipy.spatial.cKDTree(GFPpos,leafsize=10000)
            AG3_closest = np.asarray(GFP_kd.query(AG3pos,p=2,
                            #distance_upper_bound=10*psize
                            ))[0]
            AG3_500nm = AG3_closest[AG3_closest < 0.5]
            percentage = len(AG3_500nm)/AG3count*100
            print('{0},{1},{2},{3},{4},{5}'.format(_what,visit,AG3count,Fluocount,len(AG3_500nm),percentage))
            visits = np.append(visits,AG3_closest)
    all[_what] = visits
visits = np.array(visits)

SyntaxError: invalid syntax (<ipython-input-22-2e1f6b40fa19>, line 34)

In [14]:
all.keys()


# len(AG3subset['VisitP'].unique())
for a in all.keys():
    print(all[a].shape)
sns.set(font_scale = 3,style="whitegrid")

(6467,)
(11561,)
(6301,)
(16165,)
(15163,)


In [15]:
%matplotlib qt5
label = 'GFP count:'+ str(GFPcount) + '\n AG3 count:' + str(AG3count) 
fig = plt.figure()
for _cond in df['PlateRow'].unique():
    ax = sns.distplot(all[_cond], hist=False, rug=False,
#                       label=label,
                     )
ax.legend(labels=df['PlateRow'].unique())
#ax = sns.distplot(triple_closest, hist=False, rug=True, label='triple for control')
# #plt.yscale('log', nonposy='clip')
ax.set(xlabel='Distance in um', ylabel='Frequency'
#        + '  Total Particles:' + str(GFPcount)
       #,xscale="log"
       #,yscale="log"
       
      )
ax.set_xlim(0,3)
from scipy import stats
for _cond in df['PlateRow'].unique():
    print('Mode of {0}:{1}'.format(_cond,np.median(all[_cond])))
# print("Total IN:",in_count)
# print("Percentage PQBP1", pqbp1_count/float(in_count)*100, "%")
# print("Percentage Triple", triple_count/float(in_count)*100, "%")
# print("IN+cGAS without PQBP1", cgas_count/float(in_count)*100 - triple_count/float(in_count)*100 , "%")

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with si

Mode of VPRR3-D116:9.949888150812937
Mode of mNeonD116:0.20979909382521628
Mode of mWasabi-D116:0.2215293196669553
Mode of optiR3-D117:0.18834050212308073
Mode of sfGFPD117:0.2066391929327272


/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [116]:
# INsubset = viruses[viruses['Filename'] == '/run/media/jmamede/Joao/till/fixed40x/prj/WellA04_Channela647,mRuby,GFP,DAPI_Seq0003_v10_PRJ.ome.tiff']
# VCPsubset = vcps[vcps['Filename'] =='/run/media/jmamede/Joao/till/fixed40x/prj/WellA04_Channela647,mRuby,GFP,DAPI_Seq0003_v10_PRJ.ome.tiff']
# # print(INsubset['COMX mRuby'])
# INcount = INsubset.index.max()+1
# # print(VCPsubset['COMX GFP'])
# VCPcount = VCPsubset.index.max()+1
# #
# psize = 0.1081811
# # zsize = 0.125
# INpos = zip(INsubset['COMX mRuby']*psize,INsubset['COMY mRuby']*psize)

# VCPpos = zip(VCPsubset['COMX GFP']*psize,VCPsubset['COMY GFP']*psize)
# # cgas_pos = zip(cgas['Position X']*psize ,IN['Position Y']*psize ,IN['Position Z']*zsize )
# # triple_pos = zip(triple['Position X']*psize ,triple['Position Y']*psize ,triple['Position Z']*0.2)

# INpos = list(INpos)
# VCPpos = list(VCPpos)
# # zipped_list = test2[:]
# # zipped_list_2 = list(test2)
# #Nearest Neighbor Brute Force - I used 5 pixels for max distance in imaris
# IN_kd = scipy.spatial.cKDTree(INpos,leafsize=10000)
# VCP_closest = np.asarray(IN_kd.query(VCPpos,p=2,
#                 #distance_upper_bound=10*psize
#                 ))[0]


# %matplotlib inline
# label = 'VCP count:'+ str(VCPcount) + '\n IN count:' + str(INcount) 
# ax = sns.distplot(VCP_closest, hist=False, rug=True,label=label)

# #ax = sns.distplot(triple_closest, hist=False, rug=True, label='triple for control')
# #plt.yscale('log', nonposy='clip')
# ax.set(xlabel='Distance in um', ylabel='Frequency'
#        + '  Total Particles:' + str(INcount)
#        #,xscale="log"
#        #,yscale="log"
       
#       )
# ax.set_xlim(0,3)

In [177]:
import math
dual = list()
filelist = list()
print('File,AG3,Fluorophore,dual')
all = {}
for _what in df['PlateRow'].unique():
    AG3subset = df[df['PlateRow'] == _what]
    GFPsubset = df_gfp[df_gfp['PlateRow'] == _what]
    Ruby3subset = df_ruby[df_ruby['PlateRow'] == _what]
    INsubset = viruses[viruses['Filename'] == ficheiro]
    VCPsubset = vcps[vcps['Filename'] == ficheiro]

    INcount = len(INsubset)
    VCPcount = len(VCPsubset)
    


    
    psize = 0.1081811
    # zsize = 0.125

    INpos = zip(INsubset['COMX mRuby']*psize,INsubset['COMY mRuby']*psize)
    VCPpos = zip(VCPsubset['COMX GFP']*psize,VCPsubset['COMY GFP']*psize)
    INpos = list(INpos)
    VCPpos = list(VCPpos)
    if math.isnan(VCPcount) or math.isnan(INcount):
        if math.isnan(VCPcount):
            VCPcount = 0
            VCP_closest = []
        elif math.isnan(INcount):
            INcount = 0
    else:
#         print(INcount,VCPcount)
        try:
# Nearest Neighbor Brute Force - I used 5 pixels for max distance in imaris
            IN_kd = scipy.spatial.cKDTree(INpos,leafsize=10000)
            VCP_closest = np.asarray(IN_kd.query(VCPpos,p=2,k=1
        #                     distance_upper_bound=5*psize
                            ))[0]
            VCP_5pixels = VCP_closest[VCP_closest < 0.5]
            #

            print('{0},{1},{2},{3}'.format(ficheiro,INcount,VCPcount,len(VCP_5pixels)))
            dual.append(VCP_5pixels)
            filelist.append(ficheiro)
        except: print(ficheiro,'¯\_(ツ)_/¯ ')
    #in_list.append(INpos)
    
    


File,IN,VCP,dual
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v10_PRJ.ome.tiff,1314,98,78
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v1_PRJ.ome.tiff,101,5,2
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v2_PRJ.ome.tiff,149,17,9
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v3_PRJ.ome.tiff,583,68,57
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v4_PRJ.ome.tiff,291,29,20
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v5_PRJ.ome.tiff,495,43,33
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v6_PRJ.ome.tiff,793,66,56
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v7_PRJ.ome.tiff,507,53,44
/run/media/jmamede/Joao/till/fixed40x/prj/WellA01_Channela647,mRuby,GFP,DAPI_Seq0000_v8_PRJ.ome.

/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v2_PRJ.ome.tiff,18,15,12
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v3_PRJ.ome.tiff,1092,38,29
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v4_PRJ.ome.tiff,877,34,24
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v5_PRJ.ome.tiff,954,28,20
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v6_PRJ.ome.tiff,409,17,12
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v7_PRJ.ome.tiff,870,45,36
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v8_PRJ.ome.tiff,760,28,22
/run/media/jmamede/Joao/till/fixed40x/prj/WellB03_Channela647,mRuby,GFP,DAPI_Seq0007_v9_PRJ.ome.tiff,955,63,40
/run/media/jmamede/Joao/till/fixed40x/prj/WellB04_Channela647,mRuby,GFP,DAPI_Seq0006_v10_PRJ.ome.tiff,508,12,7
/

/run/media/jmamede/Joao/till/fixed40x/prj/WellC05_Channela647,mRuby,GFP,DAPI_Seq0014_v7_PRJ.ome.tiff,250,10,8
/run/media/jmamede/Joao/till/fixed40x/prj/WellC05_Channela647,mRuby,GFP,DAPI_Seq0014_v8_PRJ.ome.tiff,136,4,3
/run/media/jmamede/Joao/till/fixed40x/prj/WellC05_Channela647,mRuby,GFP,DAPI_Seq0014_v9_PRJ.ome.tiff,143,7,6
/run/media/jmamede/Joao/till/fixed40x/prj/WellC06_Channela647,mRuby,GFP,DAPI_Seq0015_v10_PRJ.ome.tiff,10,12,8
/run/media/jmamede/Joao/till/fixed40x/prj/WellC06_Channela647,mRuby,GFP,DAPI_Seq0015_v1_PRJ.ome.tiff,9,12,8
/run/media/jmamede/Joao/till/fixed40x/prj/WellC06_Channela647,mRuby,GFP,DAPI_Seq0015_v2_PRJ.ome.tiff,13,7,6
/run/media/jmamede/Joao/till/fixed40x/prj/WellC06_Channela647,mRuby,GFP,DAPI_Seq0015_v3_PRJ.ome.tiff,5,4,3
/run/media/jmamede/Joao/till/fixed40x/prj/WellC06_Channela647,mRuby,GFP,DAPI_Seq0015_v4_PRJ.ome.tiff,5,2,2
/run/media/jmamede/Joao/till/fixed40x/prj/WellC06_Channela647,mRuby,GFP,DAPI_Seq0015_v5_PRJ.ome.tiff,5,3,3
/run/media/jmamede/Joao/t

In [169]:
len(dual[0][dual[0] < 5*psize])


78

In [113]:
%matplotlib qt5

for i in range(len(filelist)):
#     label = 'VCP count:'+ str(VCPcount) + '\n IN count:' + str(INcount) 
    ax = sns.distplot(dual[i], hist=False, rug=True,label=label)

    #ax = sns.distplot(triple_closest, hist=False, rug=True, label='triple for control')
    #plt.yscale('log', nonposy='clip')
ax.set(xlabel='Distance in um', ylabel='Frequency'
       + '  Total Particles:' + str(INcount)
       #,xscale="log"
       #,yscale="log"

      )
ax.set_xlim(0,1)

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:305: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

...

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2589: RuntimeWarning: Mean of empty slice.
  line, = ax.plot(a.mean(), 0)
/home/jmamede/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seab

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2055: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/jmame

(0.0, 1.0)